In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import DateType
from datetime import datetime, timedelta
import random

spark = SparkSession.builder.getOrCreate()

# ---------------------
# 1. Load Bronze Tables
# ---------------------
df_inventory = spark.read.format("delta").load("/mnt/delta/bronze/inventory")
df_shipments = spark.read.format("delta").load("/mnt/delta/bronze/shipments")
df_vendors = spark.read.format("delta").load("/mnt/delta/bronze/vendors")

# --------------------------------------
# 2. Enrich Inventory: Add `vendor_id`
# --------------------------------------
df_inventory_enriched = df_inventory.withColumn(
    "vendor_id",
     when(col("item_id") == "ITM001", "V001")
    .when(col("item_id") == "ITM002", "V002")
    .when(col("item_id") == "ITM003", "V003")
    .when(col("item_id") == "ITM004", "V004")
    .when(col("item_id") == "ITM005", "V005")
    .when(col("item_id") == "ITM006", "V006")
    .when(col("item_id") == "ITM007", "V007")
    .when(col("item_id") == "ITM008", "V008")
    .when(col("item_id") == "ITM009", "V009")
    .otherwise("V010")
)

# -------------------------------------------------
# 3. Enrich Shipments: Add random `shipment_date`
# -------------------------------------------------
@udf(DateType())
def random_date():
    base = datetime(2025, 6, 1)
    return base + timedelta(days=random.randint(0, 14))

df_shipments_enriched = df_shipments.withColumn("shipment_date", random_date())

# ----------------------------
# 4. Write to Silver Layer
# ----------------------------
df_inventory_enriched.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/delta/silver/inventory_enriched")

df_shipments_enriched.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/delta/silver/shipments_enriched")

df_vendors.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/delta/silver/vendors_clean")

